In [1]:
"""
reads and prepares the pathway data

* Subject ids in gene expression data and PAM50 data do not exactly macth. Gene expression data has
  longer subject ids. When shortening them to match to PAM50 data, some of them collapse on the same
  id. Therfeore, we have to rectify this issue first.

* pathway-gene data uses hgnc annotation, while genexp data uses ensembl. First, find equivalent of 
  hgnc in ensembl annotation, and then sum the gene expression data to build the pathway data.
  
  
Adham Beyki
PRaDA, AA2I2 - Deakin University
2018-11-16
"""

import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from sklearn.model_selection import train_test_split

np.random.seed(100)

In [2]:
def transpose_df(df):
    
    cols = df.columns
    vals = df[cols[1:]].values
    
    df_tr = pd.DataFrame(data=np.transpose(vals), columns=df["GENE-ID"].tolist())
    df_tr["Sample-ID"] = cols[1:]
    
    return df_tr


def prep_genexp_data(genexp_path, pam50_path, sample_filter_path):
    """ Reads the data and returns a dataframe of gene expresession and
    labels of samples.
    """

    # read data files
    genexp_df = pd.read_csv(genexp_path)
    genexp_df.rename(columns={'Unnamed: 0': 'GENE-ID'}, inplace=True)
    pam50_df = pd.read_csv(pam50_path)
    sample_filter = pd.read_csv(sample_filter_path, header=None)


    # filter samples to keep PAM50 samples 
    col_idxs = sample_filter.values.astype(bool).flatten()
    col_idxs = np.hstack([np.array(True), col_idxs])    # insert a True at the beginning to keep GENE-ID column
    cols = genexp_df.columns[col_idxs]
    genexp_df = transpose_df(genexp_df[cols])    

    # remove gene ENSG00000179546 as it has NULL values for some samples
    idxs = genexp_df.isna().sum() != 0
    cols = genexp_df.columns[idxs].tolist()
    genexp_df.drop(cols, axis=1, inplace=True)

    # check if filtering the samples and their order is correct
    assert (genexp_df['Sample-ID'].apply(lambda s: '-'.join(s.split('-')[:4])[:-1]) == pam50_df["Sample ID"]).sum() == len(pam50_df)

    # only consider LumA and LumB
    idxs = pam50_df['PAM50'].apply(lambda s: s in ['LumA', 'LumB'])
    data_df = genexp_df[idxs].copy()
    data_df['PAM50'] = pam50_df[idxs]['PAM50']

    data_df.reset_index(inplace=True, drop=True)
    
    return data_df

In [3]:
# data files
GENEXP_PATH = "../data/TCGA-BRCA-htseq-ensg-normCounts.csv.gz"
PAM50_PATH = "../data/TCGA-BRCA-pam50-annot.csv"
SAMPLE_FILTER_PATH = "../data/TCGA-BRCA-htseq-ensg-colFilter.txt"

genexp_df = prep_genexp_data(GENEXP_PATH, PAM50_PATH, SAMPLE_FILTER_PATH)

<hr>

In [4]:
# prep pathways
PATHWAY_PATH = "../data/pathways/hgncToGObp.csv"
MIN_NB_GENES = 10

In [5]:
def map_pathways_to_genes(pathway_path):
    """ Reads the pathway data and returns a dictionary with pathways as keys and
    list of their contributing genes as values.
    """
    
    gene_pw_df = pd.read_csv(PATHWAY_PATH)
    pathway_to_genes = defaultdict(list)
    for i, row in gene_pw_df.iterrows():
        gene = row["SYMBOL"]
        pathways = row["GO"].split("|")
        for pathway in pathways:
            pathway_to_genes[pathway].append(gene)
    return pathway_to_genes

In [6]:
pathway_to_hgnc = map_pathways_to_genes(PATHWAY_PATH)
pathway_to_ensembl = defaultdict(list)

hgnc_to_ensembl = pd.read_pickle('gene_annotation/hgnc_to_ensembl.pkl')
pathways = list(pathway_to_hgnc.keys())
for pw in pathways:
    for g in pathway_to_hgnc[pw]:
        pathway_to_ensembl[pw] += hgnc_to_ensembl[g]

In [7]:
# filtering out genes that do not exist in genexp data
tmp = []
for pathway, genes in pathway_to_ensembl.items():
    genes2 = []
    for gene in genes:
        if gene in genexp_df.columns:
            genes2.append(gene)
    tmp.append((pathway, genes2))
pathway_to_ensembl = dict(tmp)

In [8]:
# remove pathways that have fewer genes than a threshold
tmp = []
for pw, gn in pathway_to_ensembl.items():
    if len(gn)>MIN_NB_GENES:
        tmp.append([pw, gn])
pathway_to_ensembl = dict(tmp)

In [9]:
# construct the data
pathway_df = pd.DataFrame()
for pathway, genes in tqdm(pathway_to_ensembl.items(), desc="pathway from genexp"):
    pathway_df[pathway] = genexp_df[genes].sum(1)

pathway_df["Sample-ID"] = genexp_df["Sample-ID"]
pathway_df["PAM50"] = genexp_df["PAM50"]
pathway_df.reset_index(drop=True, inplace=True)

pathway from genexp: 100%|██████████| 3942/3942 [00:06<00:00, 605.38it/s]


<hr>

In [11]:
OUTPUT = 'pathway_data.pkl'

In [12]:
# cook up X and y
cols = pathway_df.columns[:-2]
X = pathway_df[cols].values
y = pathway_df['PAM50'].values
X_train, X_test, y_train, y_test, train_idxs, test_idxs = train_test_split(X, y, np.arange(X.shape[0]), test_size=1/3, stratify=y)

pd.to_pickle(
    {
        'pathway_df': pathway_df,
        'train_idxs': train_idxs,
        'test_idxs': test_idxs
    }, OUTPUT    
)